In [2]:
import numpy as np
import random
import math
import copy
import pprint as pp
# If still humans in the place where zombies are, this will move inspite humans in cell?
# Averages, should we calculate humans or average humans in order to do the killings?

# Note: Rize pop 94 800 inhabitants

In [3]:
max_age = 15

In [4]:
cell_ = {
    'i': None, 
    'j': None, 
    'elev': None, 
    'hum': 0, 
    'z_1': [0] * max_age, 
    'z_0': [0] * max_age 
}

def initializeCell():
    return copy.deepcopy(cell_)

In [5]:
# Determine the killed humans and update its cell value
def zombiesKillHumans(cell, layer):
    h = cell['hum'] # * 255 ? is it average or what?
    z = sum(cell['z_' + str(int(layer))])
    killed_h = 10 * z
    remaining_h = h - killed_h if h > killed_h else 0
    cell['hum'] = remaining_h
    cell['z_' + str(int(layer))][0] = h - remaining_h

In [26]:
# Determine the killed zombies and update its cell value
def humanKillZombies(cell, layer):
    h = cell['hum']
    z = sum(cell['z_' + str(int(layer))])
    #print(f'to kill: {z} and {h}')
    killed_z = 10 * h
    # assign randomly to the list the killings
    print(cell['z_' + str(int(layer))])
    for k in range(killed_z):
        remaining_ages = [a for a in range(max_age) if cell['z_' + str(int(layer))][a] > 0]
        index = 0
        if len(remaining_ages) == 0:
            return
        elif len(remaining_ages) > 1:
            index = random.randint(0, len(remaining_ages) - 1)
        age = remaining_ages[index]
        cell['z_' + str(int(layer))][age] -= 1 
    print(cell['z_' + str(int(layer))])

In [7]:
# "consider cells of 15 × 15 pixels" AND "1 pixel = 1 km 2"
len_cell = 15 #km
len_diag = math.sqrt(pow(len_cell, 2) * 2) #km
dist_lin = len_cell
dist_dia = len_diag

def getGeoSlope(cell, to_cell):
    dist = dist_lin if (cell['i'] == to_cell['i'] or cell['j'] == to_cell['j']) else dist_dia
    diff = to_cell['elev'] - cell['elev']
    return math.degrees(math.atan2(diff, dist))

def getGeoSlopeFactor(cell, to_cell):
    slope = getGeoSlope(cell, to_cell)
    if slope < 0: 
        return 1 # zombies don't have any issue
    elif slope > 10:
        return 0 # zombies don't climb
    else: # then, between 0 and 10
        return 1 - (slope / 10) # zombies have certain difficulties

In [8]:
def zombiesMove(cell, neighbor_cells_list, layer):
    """ 
    1) thinking that the zombies will all go even if there are still humans
    2) We're assuming, also, that not all zombies should be moved (ex: slope constraint would encourage some to stay)
    """
    
    # today
    neighbor_humans = remaining_c = []
    for c in range(len(neighbor_cells_list)): # 8 cells max
        neighbor_humans.append(neighbor_cells_list[c]['hum'])
    h_neigh = sum(neighbor_humans)
    elev_c0 = cell['elev'] 

    # tomorrow
    autoDestroyZombies(cell, layer) # prepare zombies list for "tomorrow"
    z_c0 = sum(cell['z_' + str(int(layer))]) # zombies able to attack are just survivals aged < 15 
    #print(f'z-co: {z_c0}')
    remaining_c = [i for i in range(len(neighbor_cells_list))]
    random.shuffle(remaining_c)
    for c in range(len(neighbor_cells_list)): # 8 cells max
        #ran = random.randint(0, len(remaining_c) - 1) if len(remaining_c) > 0 else 0
        c_rand = remaining_c[-1]
        h = neighbor_cells_list[c_rand]['hum']
        geo_slope_fact = getGeoSlopeFactor(cell, neighbor_cells_list[c_rand]) # (from, to)
        z_new = getZombiesContribution(neighbor_cells_list[c_rand], h_neigh, z_c0, geo_slope_fact)
        #if layer == True : print("second")
        #if z_new > 0: print(f'z_new: {z_new}, c: {c_rand}, neigh: {len(neighbor_cells_list)}, rem: {len(remaining_c)}{remaining_c}')
        assignZombiesByAge(cell, neighbor_cells_list[c_rand], z_new, layer)
        remaining_c.pop()
    
    for a in range(max_age): # assign remaining to next layer and reinitialize old layer
        cell['z_' + str(int(not layer))][a] += cell['z_' + str(int(layer))][a] 
        cell['z_' + str(int(layer))][a] = 0

In [9]:
def getZombiesContribution(cell, h_neigh, z_c0, geo_slope_fact):
    h = cell['hum']
    #print(f'h cell: {h}, z_c0: {z_c0}, h_neigh: {h_neigh}')
    if h_neigh == 0:
        return 0
    elif h_neigh > 0:
        return math.ceil((h / h_neigh) * z_c0 * geo_slope_fact)
    else:
        print("Error in Zombies contribution: z < 0 ?")

In [10]:
def autoDestroyZombies(cell, layer):
    cell['z_' + str(int(layer))].pop() # remove old zombies (last place)
    cell['z_' + str(int(layer))].insert(0, 0) # insert space for eaten humans (first place)
    
def assignZombiesByAge(cell, new_cell, n, layer):
    for i in range(n):
        remaining_ages = [a for a in range(max_age) if cell['z_' + str(int(layer))][a] > 0]
        index = 0
        if len(remaining_ages) == 0:
            return
        elif len(remaining_ages) > 1:
            index = random.randint(0, len(remaining_ages) - 1)        
        age = remaining_ages[index]
        cell['z_' + str(int(layer))][age] -= 1
        new_cell['z_' + str(int(not layer))][age] += 1
    #print(cell['zom'])
    #print(new_cell['zom'])

In [11]:
def showMatrixVals(m, key):
    new_m = [[0 for i in range(len(m[j]))] for j in range(len(m))]
    for i in range(len(m)):
        for j in range(len(m[i])):
            new_m[i][j] = m[i][j][key]
    return new_m

In [12]:
def getNeigborCellList(i, j, m):
    n = []
    for i_sub in range(3): # Rows
        for j_sub in range(3): # Cols
            if i_sub == 1 and j_sub == 1: continue 
            if isCellCandidateForMove(i, i_sub, j, j_sub, m): 
                n.append(m[i + i_sub - 1][j + j_sub - 1])
    return n

In [13]:
def isCellCandidateForMove(i, i_sub, j, j_sub, m):
    # i: Rows, j: Columns
    ip = i + i_sub - 1
    jp = j + j_sub - 1
    if ip < 0 or jp < 0 : return False
    i_len = len(m[i]) # how many rows
    j_len = len(m) # how many columns
    if ip >= i_len or jp >= j_len: return False
    #print(f'checking i|ip|i_len: {i},{ip},{i_len} , j|jp|j_len: {j},{jp},{j_len}')
    
    # MODIFY TO DETECT BORDERS OR CELLS NOT DESIRED
    return True

In [15]:
# TEST
def initializeTest():
    m = [[initializeCell() for i in range(5)] for j in range(5)]
    elev = 0
    for i in range(5):
        for j in range(5):
            m[i][j]['elev'] = 0# elev + 1
            m[i][j]['i'] = i
            m[i][j]['j'] = j
            m[i][j]['hum'] = 100
            #elev += 1

    m[2][2]['z_' + str(0)][0] = 100
    return m


In [27]:
def runOneMoreDay(m, layer):
    # STEP 1 (Zombies move)
    for lon in range(len(m)):
        for lat in range(len(m[lon])):
            neighbor_cells = getNeigborCellList(lon, lat, m)
            zombiesMove(m[lon][lat], neighbor_cells, layer)
    # STEP 2 (Zombies Kill)
    for lon in range(len(m)):
        for lat in range(len(m[lon])):
            zombiesKillHumans(m[lon][lat], not layer) # layer, from recently moved zombies
    # STEP 3 (Humans kill)
    for lon in range(len(m)):
        for lat in range(len(m[lon])):
            humanKillZombies(m[lon][lat], not layer) # layer, from recently moved zombies

def runSimulation(d):
    m = initializeTest() # change for real M
    layer = False # start with z_0 (True in int = 1, False is 0)
    for day in range(d):
        print("D" + str(day))
        hum0 = sum([sum(e) for e in showMatrixVals(m, 'hum')])
        runOneMoreDay(m, layer)
        layer = not layer
        z_0 = sum([sum(y) for e in showMatrixVals(m, 'z_0') for y in e])
        z_1 = sum([sum(y) for e in showMatrixVals(m, 'z_1') for y in e])
        hum1 = sum([sum(e) for e in showMatrixVals(m, 'hum')])
        print(f'hum {hum0}|{hum1}, z0: {z_0}, z1: {z_1}')
        pp.pprint(showMatrixVals(m, 'hum'))
        print("-----------")
        pp.pprint(showMatrixVals(m, 'z_' + str(int(layer))))
        
        
    #pp.pprint(showMatrixVals(m, 'hum'))
    #print("-----------")
    #pp.pprint(showMatrixVals(m, 'z_0'))
    #print("-----------")
    #pp.pprint(showMatrixVals(m, 'z_1'))
   
    
runSimulation(4)



D0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[100, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[100, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[100, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[100, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[100, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[100, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[100, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[100, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[100, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[100, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[

In [16]:
"""
neigh = getNeigborCellList(2, 2, m)
cell = m[2][2]
zombiesMove(cell, neigh)
zombiesKillHumans(m[1][1])
humanKillZombies(m[1][1])
pp.pprint(showMatrixVals(m, 'hum'))
pp.pprint(showMatrixVals(m, 'zom'))
z = sum([sum(y) for e in showMatrixVals(m, 'zom') for y in e])
print(z)
"""

"\nneigh = getNeigborCellList(2, 2, m)\ncell = m[2][2]\nzombiesMove(cell, neigh)\nzombiesKillHumans(m[1][1])\nhumanKillZombies(m[1][1])\npp.pprint(showMatrixVals(m, 'hum'))\npp.pprint(showMatrixVals(m, 'zom'))\nz = sum([sum(y) for e in showMatrixVals(m, 'zom') for y in e])\nprint(z)\n"

In [17]:
layer = True
print(int(not layer)) # True in int = 1, False is 0
print("true" if 1 else "false")
mm = [[initializeCell() for i in range(10)] for j in range(5)]
elev = 0
print(len(mm))
#for i in range(len(mm)):
    #for j in range(len(mm[i])):
        #print(j)
n = np.array(mm)
print(n.shape)
pp.pprint(showMatrixVals(mm, 'hum'))
print(np.random.randint(low=0, high=8, size=0))

0
true
5
(5, 10)
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[]


In [18]:
layer = True
runOneMoreDay(m, layer)
#pp.pprint(showMatrixVals(m, 'hum'))
#print("-----------")
#pp.pprint(showMatrixVals(m, 'z_0'))
#print("-----------")
#pp.pprint(showMatrixVals(m, 'z_1'))
z = sum([sum(y) for e in showMatrixVals(m, 'z_0') for y in e])
print(z)
z = sum([sum(y) for e in showMatrixVals(m, 'z_1') for y in e])
print(z)

NameError: name 'm' is not defined

In [334]:
random.randint(0,0)

0